# Age and Gender Recognition

### github.com/rafaroman18

This project consist of classifying in real time the gender of a person and their age. We will be using a dataset of faces and a library of python to activate the camera of our laptop.

In [2]:
# Imports
import cv2 as cv
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
import keras.applications

import pathlib

First of all we will create a model to predict the gender and age of the people in the camera. We will train this model with a face dataset and we will use a pretrained model: XCEPTION. 

We will use the UTKFace Dataset, so we have to create a function to transform the images locally into a tensorflow dataset.

## MAYBE IT WOULD BE INTERESTING TO CREATE CLASSES OF AGE SO WE CAN CLASSIFY 10 CLASSES: 
* 0-10
* 20-30
* 30-40
* 40-50
* 50-60
* 60-70
* 70-80
* 80-90
* 90-100
* 100+

## KEEP IN MIND: DATA ANALYSIS

## DATA AUGMENTATION

In [ ]:
batch_size = 32
img_height = 200
img_width = 200

path = 'face_dataset/'
ds_train = tf.data.Dataset.list_files(str(pathlib.Path(path+'*.jpg')))

def process_path(file_path):
    image = tf.io.read_file(file_path)
    image = tf.image.decode_jpeg(image, channels=1)
    label = tf.strings.split(file_path, '\\')[1]
    label = tf.strings.split(label, '_')[0]# First test: only Age ":2]"
    label = tf.strings.to_number(label, out_type=tf.int64)
    
    return image, label

ds_train = ds_train.map(process_path).batch(batch_size)

In [ ]:
# https://towardsdatascience.com/building-a-multi-output-convolutional-neural-network-with-keras-ed24c7bc1178

In [ ]:
xception = keras.applications.Xception(
    weights="imagenet",
    input_shape=(img_height, img_width, 3),
    include_top=False,
)

xception.trainable = False

inputs = keras.Input(shape=(img_height, img_width, 3))

x = xception(inputs, training=False)

x = keras.layers.GlobalAveragePooling2D()(x)

x = keras.layers.Dropout(0.2)(x)

x = keras.layers.Dense(1)(x)

outputs = keras.layers.Activation('relu', name='age')(x)

model = keras.Model(inputs, outputs)
model.summary()

In [ ]:
model.compile(
    optimizer = keras.optimizers.Adam(),
    loss = keras.losses.MeanSquaredError(),
    metrics = [keras.metrics.Accuracy()]
    )

history = model.fit(ds_train, epochs=5, validation_data=ds_train)

In [4]:
face_cascade = cv.CascadeClassifier('haarcascade_frontalface_default.xml')
cap = cv.VideoCapture(0)


if not cap.isOpened():
    print("Cannot open camera")
    exit()

try:
    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()
        
        # if frame is read correctly ret is True
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break
        
        faces = face_cascade.detectMultiScale(
            frame,
            scaleFactor=1.1,
            minNeighbors=5,
            flags=cv.CASCADE_SCALE_IMAGE
        )
        
        for (x, y, w, h) in faces:
            cv.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # Display the resulting frame
        cv.imshow('frame', frame)
        if cv.waitKey(1) == ord('q'):
            break
finally:
    # Finally, release capture
    cap.release()
    cv.destroyAllWindows()